In [4]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [3]:
positive_vocab = [ 'awesome', 'outstanding', 'fantastic', 'terrific', 'good', 'nice', 'great', ':)' ]
negative_vocab = [ 'bad', 'terrible','useless', 'hate', ':(' ]
neutral_vocab = [ 'movie','the','sound','was','is','actors','did','know','words','not' ]

In [4]:
def word_feats(words):
    return dict([(word, True) for word in words])
 
positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]
neutral_features = [(word_feats(neu), 'neu') for neu in neutral_vocab]

In [5]:
train_set = negative_features + positive_features + neutral_features

In [8]:
classifier = NaiveBayesClassifier.train(train_set)

In [12]:
def word_feats(words):
    return dict([(word, True) for word in words])
 
positive_vocab = [ 'awesome', 'outstanding', 'fantastic', 'terrific', 'good', 'nice', 'great', ':)' ]
negative_vocab = [ 'bad', 'terrible','useless', 'hate', ':(' ]
neutral_vocab = [ 'movie','the','sound','was','is','actors','did','know','words','not' ]
 
positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]
neutral_features = [(word_feats(neu), 'neu') for neu in neutral_vocab]
 
train_set = negative_features + positive_features + neutral_features
 
classifier = NaiveBayesClassifier.train(train_set) 
 
# Predict
neg = 0
pos = 0
sentence = "bad movie, I liked it"
sentence = sentence.lower()
words = sentence.split(' ')
for word in words:
    classResult = classifier.classify( word_feats(word))
    if classResult == 'neg':
        neg = neg + 1
    if classResult == 'pos':
        pos = pos + 1
 
print('Positive: ' + str(float(pos)/len(words)))
print('Negative: ' + str(float(neg)/len(words)))

Positive: 0.4
Negative: 0.4


In [7]:

EXAMPLE_TEXT = "Hello Mr. Smith, how are you doing today? The weather is great, and Python is awesome. The sky is pinkish-blue. You shouldn't eat cardboard."
print(sent_tokenize(EXAMPLE_TEXT))


['Hello Mr. Smith, how are you doing today?', 'The weather is great, and Python is awesome.', 'The sky is pinkish-blue.', "You shouldn't eat cardboard."]


In [16]:
data = pd.read_csv("replay-question-data-prod.csv")
replayDataframe = pd.DataFrame(data)
print (replayDataframe)


                                                  Text
0                          Who is the director of TLT?
1                                Runways are numbered:
2    If displaced threshold markings are indicated ...
3                       In regard to runway lighting: 
4    The FAA is installing pilot control of lights ...
5    If you are on the glide slope of a 2 Bar VASI ...
6    If you are above the glide path while using a ...
7          What is the indication for a closed runway?
8    For a lighted land airport you would expect a ...
9         As the gross weight of an airplane increases
10   If the airplane is loaded so that the center o...
11                What is the definition of the Datum?
12        The basic formula for weight and balance is:
13   What is the weight of standard aviation gasoli...
14           What does Basic Empty Weight not include?
15   The condition which causes the most severe win...
16   If you are approaching to land behind a landin...
17   Wake 

In [23]:
import PIL
import itertools
import matplotlib.pyplot as plt
from wordcloud import WordCloud
 
 
reviews_names = ['animals','Blood','atmospheric','formula','TLT']
for reviews,name in zip(frames,reviews_names):
    raw_str = replayDataframe.loc[Text].str.cat(sep=',')
    wordcloud = WordCloud( max_words=1000,margin=0).generate(raw_str)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()
 

NameError: name 'frames' is not defined

In [26]:
replayDataframe['tokenized_sents'] = replayDataframe.apply(lambda row: nltk.word_tokenize(row['Text']), axis=1)

print(replayDataframe)



                                                  Text  \
0                          Who is the director of TLT?   
1                                Runways are numbered:   
2    If displaced threshold markings are indicated ...   
3                       In regard to runway lighting:    
4    The FAA is installing pilot control of lights ...   
5    If you are on the glide slope of a 2 Bar VASI ...   
6    If you are above the glide path while using a ...   
7          What is the indication for a closed runway?   
8    For a lighted land airport you would expect a ...   
9         As the gross weight of an airplane increases   
10   If the airplane is loaded so that the center o...   
11                What is the definition of the Datum?   
12        The basic formula for weight and balance is:   
13   What is the weight of standard aviation gasoli...   
14           What does Basic Empty Weight not include?   
15   The condition which causes the most severe win...   
16   If you ar

In [27]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [30]:
from bs4 import BeautifulSoup
html_str = '''
<td><a href="http://www.fakewebsite.com">Please can you strip me?</a>
<br/><a href="http://www.fakewebsite.com">I am waiting....</a>
</td>
'''
soup = BeautifulSoup(html_str, "lxml")

print(soup.get_text())

Please can you strip me?
I am waiting....




In [32]:
list(replayDataframe)

['Text', 'tokenized_sents']

In [106]:
#Sentimental Analyzer

sentiAna = SentimentIntensityAnalyzer()
results = []
for line in replayDataframe['tokenized_sents']:
    pol_score = sentiAna.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

df = pd.DataFrame.from_records(results)
df.head(50)


,compound,headline,neg,neu,pos
0,0.0000,Who is the director of TLT?,0.000,1.000,0.000
1,0.0000,Runways are numbered:,0.000,1.000,0.000
2,0.0000,If displaced threshold markings are indicated ...,0.000,1.000,0.000
3,0.0000,In regard to runway lighting:,0.000,1.000,0.000
4,0.0000,The FAA is installing pilot control of lights ...,0.000,1.000,0.000
5,0.0000,If you are on the glide slope of a 2 Bar VASI ...,0.000,1.000,0.000
6,0.0000,If you are above the glide path while using a ...,0.000,1.000,0.000
7,0.0000,What is the indication for a closed runway?,0.000,1.000,0.000
8,0.0000,For a lighted land airport you would expect a ...,0.000,1.000,0.000
9,-0.4767,As the gross weight of an airplane increases,0.307,0.693,0.000


In [177]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


#Remove html
replayDataframe['tokenized_sents'] = replayDataframe['Text'].map(lambda Text: BeautifulSoup(Text, "lxml").get_text())


#Tokenize Sent/word

replayDataframe['tokenized_sents'] = replayDataframe.apply(lambda row: nltk.sent_tokenize(row['tokenized_sents']), axis=1)




In [178]:
print(replayDataframe['tokenized_sents'])


0                          [Who is the director of TLT?]
1                                [Runways are numbered:]
2      [If displaced threshold markings are indicated...
3                        [In regard to runway lighting:]
4      [The FAA is installing pilot control of lights...
5      [If you are on the glide slope of a 2 Bar VASI...
6      [If you are above the glide path while using a...
7          [What is the indication for a closed runway?]
8      [For a lighted land airport you would expect a...
9         [As the gross weight of an airplane increases]
10     [If the airplane is loaded so that the center ...
11                [What is the definition of the Datum?]
12        [The basic formula for weight and balance is:]
13     [What is the weight of standard aviation gasol...
14           [What does Basic Empty Weight not include?]
15     [The condition which causes the most severe wi...
16     [If you are approaching to land behind a landi...
17     [Wake turbulence (wing-t

In [142]:
#stop word 

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

filtered_question = []
#for stopword in replayDataframe['tokenized_sents'].str:
#    if stopword in stop_word:
#       filtered_question.append(stopword)
        
replayDataframe['filtered_sentence_without_stopwords'] = replayDataframe['tokenized_sents'].apply(lambda tokenized_sents: ' '.join([word for word in tokenized_sents if not word in (stop_words)]))
#filtered_sentence = [w for w in replayDataframe['tokenized_sents'] if not w in stop_words]

print(replayDataframe['filtered_sentence_without_stopwords'])

0                                     Who director TLT ?
1                                     Runways numbered :
2      If displaced threshold markings indicated runw...
3                            In regard runway lighting :
4      The FAA installing pilot control lights airpor...
5                  If glide slope 2 Bar VASI would see :
6      If glide path using 4 light Precision Approach...
7                        What indication closed runway ?
8      For lighted land airport would expect rotating...
9                     As gross weight airplane increases
10     If airplane loaded center gravity located forw...
11                               What definition Datum ?
12                    The basic formula weight balance :
13     What weight standard aviation gasoline ( AvGas...
14                     What Basic Empty Weight include ?
15          The condition causes severe wingtip vortices
16     If approaching land behind landing large aircr...
17          Wake turbulence ( w

In [143]:
#Text Classification
#replayDataframe['filtered_sentence'] = replayDataframe.apply(lambda row: nltk.word_tokenize(row['tokenized_sents']), axis=1)

all_words = nltk.FreqDist(replayDataframe['filtered_sentence_without_stopwords'])

print(all_words)
print(all_words.most_common(15))

<FreqDist with 764 samples and 1000 outcomes>
[('The Replay leaderboard shows true quiz scores learners .', 15), ('What types quizzes create Replay ?', 15), ('Achievements Replay help learners gain experience points ( XP ) level .', 15), ('Replay :', 15), ('Your avatar Replay anonymous peers .', 15), ('When Purdue Memorial Union open ?', 4), ('Of Purdue Memorial Union memoriam ? Students :', 4), ('Which building remains original six-building campus ?', 4), ('For class ’ 50th anniversary Engineering Fountain dedicated ? ( Fun fact : It happened Homecoming )', 4), ('What part Engineering Fountain added 2001 ?', 4), ('How many Purdue regional campuses located throughout state Indiana ?', 4), ('How many colleges/schools Purdue University consist ?', 4), ('How many Interfraternity Council Fraternities Purdue ’ campus ?', 4), ('Purdue ’ Recreational Center named former Purdue president ?', 4), ('Who first president Purdue ?', 4)]


In [204]:
#Tokenize Sent/word

replayDataframe['tokenized'] = replayDataframe.apply(lambda row: nltk.word_tokenize(row['filtered_sentence_without_stopwords']), axis=1)

print(replayDataframe['tokenized'])



0                                [Who, director, TLT, ?]
1                                 [Runways, numbered, :]
2      [If, displaced, threshold, markings, indicated...
3                      [In, regard, runway, lighting, :]
4      [The, FAA, installing, pilot, control, lights,...
5        [If, glide, slope, 2, Bar, VASI, would, see, :]
6      [If, glide, path, using, 4, light, Precision, ...
7                  [What, indication, closed, runway, ?]
8      [For, lighted, land, airport, would, expect, r...
9               [As, gross, weight, airplane, increases]
10     [If, airplane, loaded, center, gravity, locate...
11                          [What, definition, Datum, ?]
12             [The, basic, formula, weight, balance, :]
13     [What, weight, standard, aviation, gasoline, (...
14              [What, Basic, Empty, Weight, include, ?]
15     [The, condition, causes, severe, wingtip, vort...
16     [If, approaching, land, behind, landing, large...
17     [Wake, turbulence, (, wi

In [205]:
from collections import Counter

values = ','.join([str(i) for i in replayDataframe['tokenized']])

print(values)

all_question_replay = nltk.FreqDist(values)

rslt = pd.DataFrame(all_question_replay.most_common(50),
                    columns=['Word', 'Frequency'])

rslt = pd.DataFrame(Counter(values).most_common(50),
columns=['Word', 'Frequency']).set_index('Word')
print(all_question_replay)
print(rslt)
print(all_question_replay.most_common(50))

['Who', 'director', 'TLT', '?'],['Runways', 'numbered', ':'],['If', 'displaced', 'threshold', 'markings', 'indicated', 'runway', ',', 'know', 'part', 'runway', 'leading', 'displaced', 'threshold', ':'],['In', 'regard', 'runway', 'lighting', ':'],['The', 'FAA', 'installing', 'pilot', 'control', 'lights', 'airports', 'control', 'tower', 'F.S.S', '.', 'With', 'regard', 'system', ',', 'know', ':'],['If', 'glide', 'slope', '2', 'Bar', 'VASI', 'would', 'see', ':'],['If', 'glide', 'path', 'using', '4', 'light', 'Precision', 'Approach', 'Path', 'Indicator', '(', 'PAPI', ')', ',', 'would', 'see', 'colors', '?'],['What', 'indication', 'closed', 'runway', '?'],['For', 'lighted', 'land', 'airport', 'would', 'expect', 'rotating', 'beacon', 'would', 'alternating', '_________', '________', 'colored', 'lights'],['As', 'gross', 'weight', 'airplane', 'increases'],['If', 'airplane', 'loaded', 'center', 'gravity', 'located', 'forward', 'established', 'center', 'gravity', 'limits'],['What', 'definition', '

In [231]:


def join_strings(words):
    result=""
    for word in words:
        result.append(word)
    return result
df_one = ','.join([str(i) for i in replayDataframe['tokenized']])

classifier = nltk.NaiveBayesClassifier.train(df_one)

ValueError: not enough values to unpack (expected 2, got 1)

In [238]:
data_train = pd.read_csv('train_set.csv')
replay_train = pd.DataFrame(data)



UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 0-1: invalid continuation byte